## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-19-15-06-13 +0000,nyt,Live Updates: Former Prince Andrew Arrested in...,https://www.nytimes.com/live/2026/02/19/world/...
1,2026-02-19-15-03-47 +0000,nyt,"U.S. Imports Grew in 2025, as Trump’s Tariffs ...",https://www.nytimes.com/2026/02/19/business/ec...
2,2026-02-19-15-02-00 +0000,wsj,Dollar Gets Support From Prospect of Steady U....,https://www.wsj.com/finance/currencies/asian-c...
3,2026-02-19-15-00-05 +0000,startribune,Andrew Mountbatten-Windsor arrested on suspici...,https://www.startribune.com/andrew-mountbatten...
4,2026-02-19-14-58-00 +0000,nyt,Trump Mulls a North American Trade Pact Withou...,https://www.nytimes.com/2026/02/19/business/ec...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2428/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
11,trump,30
7,epstein,17
82,iran,15
4,andrew,15
374,california,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
84,2026-02-19-11-48-00 +0000,wsj,"As Trump’s Board of Peace meets Thursday, can ...",https://www.wsj.com/politics/trump-moves-towar...,115
331,2026-02-18-19-06-00 +0000,wsj,NIH Director Dr. Jay Bhattacharya will step in...,https://www.wsj.com/politics/policy/rfk-jr-cdc...,85
68,2026-02-19-12-11-49 +0000,nypost,UK Prime Minister Keir Starmer warned ‘nobody ...,https://nypost.com/2026/02/19/world-news/uk-pr...,84
168,2026-02-19-07-49-29 +0000,nypost,Former South Korean President Yoon sentenced t...,https://nypost.com/2026/02/19/world-news/forme...,84
0,2026-02-19-15-06-13 +0000,nyt,Live Updates: Former Prince Andrew Arrested in...,https://www.nytimes.com/live/2026/02/19/world/...,80


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
84,115,2026-02-19-11-48-00 +0000,wsj,"As Trump’s Board of Peace meets Thursday, can ...",https://www.wsj.com/politics/trump-moves-towar...
68,84,2026-02-19-12-11-49 +0000,nypost,UK Prime Minister Keir Starmer warned ‘nobody ...,https://nypost.com/2026/02/19/world-news/uk-pr...
168,61,2026-02-19-07-49-29 +0000,nypost,Former South Korean President Yoon sentenced t...,https://nypost.com/2026/02/19/world-news/forme...
3,59,2026-02-19-15-00-05 +0000,startribune,Andrew Mountbatten-Windsor arrested on suspici...,https://www.startribune.com/andrew-mountbatten...
13,57,2026-02-19-14-39-21 +0000,nypost,White House warns Iran it’d be ‘very wise’ to ...,https://nypost.com/2026/02/19/us-news/white-ho...
324,56,2026-02-18-19-30-00 +0000,cbc,"8 skiers found dead, 1 still missing in deadli...",https://www.cbc.ca/news/world/california-avala...
331,55,2026-02-18-19-06-00 +0000,wsj,NIH Director Dr. Jay Bhattacharya will step in...,https://www.wsj.com/politics/policy/rfk-jr-cdc...
139,54,2026-02-19-09-58-53 +0000,nypost,Rep. Lauren Boebert’s son Tyler slapped with a...,https://nypost.com/2026/02/19/us-news/lauren-b...
187,49,2026-02-19-05-01-09 +0000,nyt,Policy Flip-Flops Hurt the British Leader. The...,https://www.nytimes.com/2026/02/19/world/europ...
322,44,2026-02-18-19-43-00 +0000,wsj,Fed officials signaled little appetite for red...,https://www.wsj.com/economy/central-banking/fe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
